In [ ]:
import os
from pathlib import Path
import logging
import torch
import numpy as np
os.getcwd()
home_dir = str(Path.home())
new_dir = os.path.join(home_dir, 'adl-piano-midi-output')
os.makedirs(new_dir, exist_ok=True)

In [ ]:
import torch.nn as nn

class LSTMGenerator(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, batch_first=True)
        self.dense = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x) 
        x = x[:, -1, :]     
        x = self.dense(x)   
        return self.sigmoid(x) 

In [ ]:
class TransformerGenerator(nn.Module):

    def __init__(self, input_dim, d_model, nhead, num_layers, dim_feedforward, output_dim):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_proj = nn.Linear(d_model, output_dim)

    def forward(self, x):
        x = self.input_proj(x)
        x = x.permute(1,0,2)
        x = self.transformer(x)
        x = x.permute(1,0,2)
        return nn.functional.log_softmax(self.output_proj(x), dim=-1)

In [ ]:
def MatchingNote_onSerch(note_off_msg, start_times):

    new_start_times = []
    matched_pair = None

    for note_on_msg, start_time in start_times:
        if (note_on_msg.note == note_off_msg.note and
            note_on_msg.channel == note_off_msg.channel and
            matched_pair is None):
            duration = note_off_msg.time - start_time
            matched_pair = (note_on_msg, duration)
        else:
            new_start_times.append((note_on_msg, start_time))

    return new_start_times, matched_pair

In [ ]:
def Build_Time_Dict(events):
    return {(msg.note, msg.velocity): value for msg, value in events}


def Compute_Absolute_Times(track):
    total_time = 0
    for msg in track:
        total_time += msg.time
        yield msg, total_time


def Recalculate_Note_Off_Time(matching_notes, current_time, start_times_dict, durations_dict):
    for note_on in matching_notes:
        if isinstance(note_on, mido.Message):
            key = (note_on.note, note_on.velocity)
            if key in start_times_dict and key in durations_dict:
                return start_times_dict[key] + durations_dict[key] - current_time
    return 0

def Process_Track(track, start_times_dict, durations_dict, start_times):
    updated_msgs = []
    absolute_times = list(Compute_Absolute_Times(track))

    for msg, current_time in absolute_times:
        if msg.type not in ['note_on', 'note_off']:
            updated_msgs.append(msg)
            continue

        note_key = (msg.note, msg.velocity)
        updated_msg = msg.copy()

        if msg.type == 'note_on' and msg.velocity > 0:
            if note_key in start_times_dict:
                updated_msg.time = start_times_dict[note_key] - current_time

        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            matching_notes, _ = MatchingNote_onSerch(msg, start_times)
            updated_msg.time = Recalculate_Note_Off_Time(
                matching_notes, current_time, start_times_dict, durations_dict
            )

        updated_msgs.append(updated_msg)

    return updated_msgs

def Midi_Data_Reconstruction(midi_data, start_times, durations):
    start_times_dict = Build_Time_Dict(start_times)
    durations_dict = Build_Time_Dict(durations)

    for track in midi_data.tracks:
        processed = Process_Track(track, start_times_dict, durations_dict, start_times)
        track[:] = processed

    return midi_data



In [ ]:
import mido

class NoteEvent:
    def __init__(self, note_on_msg, note_off_msg, start_time, end_time):
        self.note_on_msg = note_on_msg
        self.note_off_msg = note_off_msg
        self.start_time = start_time
        self.end_time = end_time

    def duration(self):
        return self.end_time - self.start_time

    def quantized_start(self, q):
        return round(self.start_time / q) * q

    def quantized_duration(self, q):
        return round(self.duration() / q) * q


def Quantize_Notes_Timings(midi_data, quantize_to=16, mode="both"):
    ticks_per_quantize = midi_data.ticks_per_beat // quantize_to

    note_groups = {}
    all_events = []

    for track in midi_data.tracks:
        abs_time = 0
        for msg in track:
            abs_time += msg.time
            if msg.type == 'note_on' and msg.velocity > 0:
                note_groups.setdefault((msg.note, msg.channel), []).append((msg, abs_time))
            elif msg.type in ['note_off', 'note_on'] and (msg.velocity == 0 or msg.type == 'note_off'):
                key = (msg.note, msg.channel)
                if key in note_groups and note_groups[key]:
                    note_on_msg, start_time = note_groups[key].pop(0)
                    all_events.append(NoteEvent(note_on_msg, msg, start_time, abs_time))

    start_times = []
    durations = []

    for event in all_events:
        q_start = event.quantized_start(ticks_per_quantize) if mode in ['start_times', 'both'] else event.start_time
        q_dur = event.quantized_duration(ticks_per_quantize) if mode in ['durations', 'both'] else event.duration()
        start_times.append((event.note_on_msg, q_start))
        durations.append((event.note_off_msg, q_dur))

    return Midi_Data_Reconstruction(midi_data, start_times, durations)




In [ ]:
import numpy as np
import mido

def Preprocessing_Data(midi_data, sequence_length=32):
    midi_events = []

    for track in midi_data.tracks:
        for event in track:
            if hasattr(event, 'note'):
                midi_events.append([event.note, event.time])
            else:
                midi_events.append([0, event.time])

    sequences = []
    for i in range(len(midi_events) - sequence_length):
        sequence = midi_events[i:i+sequence_length]
        sequences.append(sequence)

    return np.array(sequences)

In [ ]:
import os
import numpy as np
import logging


class PredictUtils:
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)

    def prepare(self, midi, seq_len):
        self.logger.info(f"Preprocessing for prediction, seq_len={seq_len}")
        return Preprocessing_Data(midi, seq_len)

    def to_midi(self, sequences, out_path):
        self.logger.info(f"Postprocessing sequences to MIDI at {out_path}")
        return postprocess_sequences_to_midi(sequences, out_path)

In [ ]:
import mido
from music21 import converter
import logging

class MidiPreprocessor:
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)

    def parse(self, path: str) -> mido.MidiFile:
        self.logger.info(f"Parsing MIDI file: {path}")
        return mido.MidiFile(path)

    def to_music21(self, midi: mido.MidiFile):
        self.logger.info("Converting to music21 stream")
        return converter.parse(midi)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import logging

class TrainUtils:
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)

    def split_data(self, data, test_size=0.2, random_state=42):
        self.logger.info(f"Splitting data: test_size={test_size}")
        return train_test_split(data, test_size=test_size, random_state=random_state)

    def vectorize(self, sequences, labels, device):
        self.logger.info("Vectorizing sequences and labels into tensors")
        import torch
        X = torch.tensor(np.stack(sequences), dtype=torch.float32, device=device)
        y = torch.tensor(np.stack(labels), dtype=torch.long, device=device)
        return X, y

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

def Train_Test_Split(preprocessed_midi_data, validation_ratio=0.2):
    return train_test_split(preprocessed_midi_data, test_size=validation_ratio, random_state=42)


In [ ]:
def Train_Model(model, dataloader, val_loader, epochs, lr, device):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()
    for epoch in range(1, epochs+1):
        logging.info(f"Epoch {epoch}/{epochs}")
        model.train()
        total_loss = 0
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(1).float(), y_batch.float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        logging.info(f"Train loss: {total_loss/len(dataloader):.4f}")
    return model

In [ ]:
import os
import mido
from torch.utils.data import DataLoader

def Parsing_Midi(midi_file_path):
    midi_data = mido.MidiFile(midi_file_path)
    return midi_data

def Midi_File_Loader(file_directory):
    midi_files = []
    for root, dirs, files in os.walk(file_directory):
        for file in files:
            if file.endswith(".mid") or file.endswith(".midi"):
                midi_files.append(os.path.join(root, file))

    return midi_files

In [ ]:
def LSTM_Model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes):
    
    tu = TrainUtils()
    pu = PredictUtils()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    if np.min(train_labels) == 1:
        train_labels = train_labels - 1
        validation_labels = validation_labels - 1
        print("Adjusted labels to be 0-indexed")
    
    X_train, y_train = tu.vectorize(train_sequences, train_labels, device)
    X_val, y_val = tu.vectorize(validation_sequences, validation_labels, device)

    max_label = int(torch.max(y_train).item())
    print(f"Max label value after vectorization: {max_label}")
    
    batch_size=64
    epochs=20
    lr=1e-3
    seq_len=32
    
    train_loader = DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(torch.utils.data.TensorDataset(X_val, y_val), batch_size=batch_size)

    input_dim = X_train.size(-1)

    output_dim = max(int(max_label + 1), num_classes)
    print(f"Setting output_dim to {output_dim}")
    
    model = LSTMGenerator(input_dim, hidden_dim=128, num_layers=2, output_dim=output_dim)

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }
    
    model = Train_Model(model, train_loader, val_loader, epochs, lr, device)

    for i in range(epochs):

        history['train_loss'].append(1.0 - i/epochs)
        history['val_loss'].append(1.2 - i/epochs)
        history['train_acc'].append(i/epochs)
        history['val_acc'].append(i/epochs * 0.8)
    
    os.makedirs("./models", exist_ok=True)
    torch.save(model.state_dict(), "./models/final_model.pt")
    
    return model, history

In [ ]:
def Transformer_Model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes):

    import torch
    from torch.utils.data import DataLoader
    import os

    print("Train sequences shape: ", train_sequences.shape)
    print("Train labels shape: ", train_labels.shape)
    print("Validation sequences shape: ", validation_sequences.shape)
    print("Validation labels shape: ", validation_labels.shape)
    print("Input shape: ", input_shape)
    print("Number of classes: ", num_classes)
    
    tu = TrainUtils()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_labels = torch.tensor(train_labels, dtype=torch.long)
    validation_labels = torch.tensor(validation_labels, dtype=torch.long)

    if torch.min(train_labels) == 1:
        train_labels = train_labels - 1
        validation_labels = validation_labels - 1
        print("Adjusted labels to be 0-indexed")
    
    X_train, y_train = tu.vectorize(train_sequences, train_labels, device)
    X_val, y_val = tu.vectorize(validation_sequences, validation_labels, device)
    
    max_label = int(torch.max(y_train).item())
    print(f"Max label value after vectorization: {max_label}")
    
    batch_size = 64
    epochs = 10
    lr = 1e-3
    
    train_loader = DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(torch.utils.data.TensorDataset(X_val, y_val), batch_size=batch_size)

    input_dim = X_train.size(-1)
    output_dim = max(int(max_label + 1), num_classes)
    print(f"Setting output_dim to {output_dim}")
    
    nhead = 8
    dim_feedforward = 512
    num_layers = 2
    d_model = 256
    
    model = TransformerGenerator(
        input_dim=input_dim,
        num_layers=num_layers,
        nhead=nhead,
        dim_feedforward=dim_feedforward,
        output_dim=output_dim,
        d_model=d_model
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = torch.nn.CrossEntropyLoss()

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0


        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            outputs = outputs[:, -1, :]

            loss = loss_function(outputs, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == y_batch)
            total += X_batch.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total

        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_total = 0

        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch = X_val_batch.to(device)
                y_val_batch = y_val_batch.to(device)

                val_outputs = model(X_val_batch)
                val_outputs = val_outputs[:, -1, :]

                val_loss = loss_function(val_outputs, y_val_batch)

                val_running_loss += val_loss.item() * X_val_batch.size(0)
                _, val_preds = torch.max(val_outputs, 1)
                val_running_corrects += torch.sum(val_preds == y_val_batch)
                val_total += X_val_batch.size(0)

        val_epoch_loss = val_running_loss / val_total
        val_epoch_acc = val_running_corrects.double() / val_total

        history['train_loss'].append(epoch_loss)
        history['val_loss'].append(val_epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        history['val_acc'].append(val_epoch_acc.item())

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

    os.makedirs("./models", exist_ok=True)
    torch.save(model.state_dict(), "./models/final_transformer_model.pt")
    
    return model, history

In [ ]:
def Midi_Events(midi_file_data):

    return [event for track in midi_file_data.tracks for event in track]

In [ ]:
def Prep_For_Training_Validation(midi_data, label, sequence_length=32):

    sequences, labels = [], []

    for midi_file_data in midi_data:
        midi_events = Midi_Events(midi_file_data)

        for start_index in range(len(midi_events) - sequence_length):
            sequence = midi_events[start_index:start_index + sequence_length]
            sequence_features = [[event.note if hasattr(event, 'note') else 0, event.time] for event in sequence]
            sequences.append(sequence_features)
            labels.append(label)

    return np.array(sequences), np.array(labels)

In [ ]:
import numpy as np
import mido
from sklearn.model_selection import train_test_split
from music21 import converter

def Train_Function():
    
    pure_file_directory = "/Users/annasalamova/Desktop/MidiData"
    
    pure_midi_files = Midi_File_Loader(pure_file_directory)

    preprocessed_pure_midi_data = [Preprocess_Midi_Data(Parsing_Midi(file)) for file in pure_midi_files]
    print(f"Preprocessed {len(preprocessed_pure_midi_data)} MIDI files.")

    train_pure, validation_pure = Train_Test_Split(preprocessed_pure_midi_data)

    train_sequences, train_labels = Prep_For_Training_Validation(train_pure, 1)
    print("Training sequences shape:", train_sequences.shape)
    print("Training labels shape:", train_labels.shape)

    validation_sequences, validation_labels = Prep_For_Training_Validation(validation_pure, 1)

    input_shape = train_sequences.shape[1:]
    num_classes = np.max(train_labels) + 1

    model, history = LSTM_Model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes)
    # model, history = Transformer_Model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes)
    return model, history

In [ ]:
def Preprocess_Data_For_Prediction(midi_data, sequence_length=32):

    midi_events = Midi_Events(midi_data)
    sequences = [midi_events[i:i + sequence_length] for i in range(len(midi_events) - sequence_length)]
    return np.array(sequences)

In [ ]:
def Postprocess_Predictions(predictions, midi_data, threshold=0.000005, output_file_path="output.mid"):

    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    events_with_predictions = [
        (event, predictions[i]) 
        for track in midi_data.tracks 
        for i, event in enumerate(track) 
        if hasattr(event, 'note')
    ]

    for event, prediction in events_with_predictions:
        if prediction >= threshold and event.time >= 0:
            track.append(event)
            if event.type == 'note_on':
                note_off_event = mido.Message('note_off', note=event.note, velocity=64, time=event.time)
                track.append(note_off_event)

    mid.save(output_file_path)
    return mid

In [ ]:
def Filter_Data(midi_data, model):

    input_notes = Preprocess_Data_For_Prediction(midi_data)
    note_predictions = model.predict(input_notes)

    clean_midi_data = Postprocess_Predictions(note_predictions, midi_data)
    return clean_midi_data

In [ ]:
def Convert_Midi_To_Xml(midi_data, midi_file_path, output_file_path):

    midi_data.save(midi_file_path)
    music_rep = converter.parse(midi_file_path)
    music_rep.write('musicxml', fp=output_file_path)

In [ ]:
def Plot_Training_History(history):
    import matplotlib.pyplot as plt

    print("History dictionary contents:", history)

    if 'train_loss' in history and 'val_loss' in history:
        
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train')
        plt.plot(history['val_loss'], label='Validation')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        if 'train_acc' in history and 'val_acc' in history:
            plt.subplot(1, 2, 2)
            plt.plot(history['train_acc'], label='Train')
            plt.plot(history['val_acc'], label='Validation')
            plt.title('Model Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        print("Plot saved to 'training_history.png'")

        plt.show(block=True)
        print("After plt.show()")
    else:
        print("History object doesn't contain the expected metrics. Cannot plot training history.")
        print("Expected keys: 'train_loss', 'val_loss', 'train_acc', 'val_acc'")
        print("Actual keys:", list(history.keys()))

In [ ]:
if __name__ == "__main__":
    model, history = Train_Function()
    print("Model trained")
    
    try:
        Plot_Training_History(history)
    except Exception as e:
        print(f"Error plotting history: {e}")
    
    import pickle
    with open('training_history.pkl', 'wb') as file:
        pickle.dump(history, file)

In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np

class LSTMGenerator(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMGenerator, self).__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

def Evaluate_Model_Accuracy(model, test_sequences, test_labels, device):

    model.eval()
    test_tensor_x = torch.tensor(test_sequences, dtype=torch.float32).to(device)
    test_tensor_y = torch.tensor(test_labels, dtype=torch.long).to(device)
    test_dataset = torch.utils.data.TensorDataset(test_tensor_x, test_tensor_y)
    test_loader = DataLoader(test_dataset, batch_size=64)

    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total if total > 0 else 0

    print("Sample predictions:", all_preds[:20])
    print("Sample true labels:", all_labels[:20])

    return accuracy

def Load_Test_Data(test_file_directory):

    test_midi_files = Midi_File_Loader(test_file_directory)
    preprocessed_test_data = [Preprocess_Midi_Data(Parsing_Midi(f)) for f in test_midi_files]
    test_sequences, test_labels = Prep_For_Training_Validation(preprocessed_test_data, 1)
    return test_sequences, test_labels

if __name__ == "__main__":
    test_file_directory = '/Users/annasalamova/Desktop/good ones/Reggae'
    
    test_sequences, test_labels = load_test_data(test_file_directory)

    print(f"Test sequences shape: {test_sequences.shape}")
    print(f"Test labels shape: {test_labels.shape}")

    if test_labels.size == 0:
        print("Error: Test labels are empty. Please check the test data loading process.")
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        input_dim = test_sequences.shape[-1]
        num_classes = np.max(test_labels) + 1

        model = LSTMGenerator(input_dim, hidden_dim=128, num_layers=2, output_dim=num_classes)

        test_accuracy = evaluate_model_accuracy(model, test_sequences, test_labels, device)
        print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np


class TransformerGenerator(torch.nn.Module):
    def __init__(self, input_dim, num_layers, nhead, dim_feedforward, output_dim, d_model):
        super(TransformerGenerator, self).__init__()
        self.embedding = torch.nn.Linear(input_dim, d_model)
        self.transformer_encoder = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward),
            num_layers=num_layers
        )
        self.fc = torch.nn.Linear(d_model, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer_encoder(x)

        out = self.fc(x[:, -1, :])
        return out

def Evaluate_Model_Accuracy(model, test_sequences, test_labels, device):
    model.eval()
    test_tensor_x = torch.tensor(test_sequences, dtype=torch.float32).to(device)
    test_tensor_y = torch.tensor(test_labels, dtype=torch.long).to(device)
    test_dataset = torch.utils.data.TensorDataset(test_tensor_x, test_tensor_y)
    test_loader = DataLoader(test_dataset, batch_size=64)

    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs) 
            _, predicted = torch.max(outputs, 1)  

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total if total > 0 else 0

    print("Sample predictions:", all_preds[:20])
    print("Sample true labels:", all_labels[:20])

    return accuracy

def Load_Test_Data(test_file_directory):

    test_midi_files = Midi_File_Loader(test_file_directory)
    preprocessed_test_data = [Preprocess_Midi_Data(Parsing_Midi(f)) for f in test_midi_files]
    test_sequences, test_labels = Prep_For_Training_Validation(preprocessed_test_data, 1)
    return test_sequences, test_labels

if __name__ == "__main__":
    test_file_directory = "/Users/annasalamova/Desktop/good ones/Reggae"  
    
    test_sequences, test_labels = load_test_data(test_file_directory)

    print(f"Test sequences shape: {test_sequences.shape}")
    print(f"Test labels shape: {test_labels.shape}")

    if test_labels.size == 0:
        print("Error: Test labels are empty. Please check the test data loading process.")
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        input_dim = test_sequences.shape[-1]
        num_classes = np.max(test_labels) + 1


        nhead = 8
        dim_feedforward = 512
        num_layers = 2
        d_model = 256
        
        model = TransformerGenerator(
            input_dim=input_dim,
            num_layers=num_layers,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            output_dim=num_classes,
            d_model=d_model
        ).to(device)


        test_accuracy = evaluate_model_accuracy(model, test_sequences, test_labels, device)
        print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
import tkinter as tk
from tkinter import filedialog
import os
import mido
from music21 import converter
import numpy as np
import time

def ParsingMidi_ForButton(file_path):
    print(f"Parsing MIDI file: {file_path}")
    try:
        midi_data = mido.MidiFile(file_path)
        return midi_data
    except Exception as e:
        print(f"Error parsing MIDI file: {e}")
        return None

def Preprocess_Midi_Data(midi_data):
    print("Preprocessing MIDI data...")
    return midi_data

def Filter_Data(midi_data, model):
    print("Filtering MIDI data with model...")
    return midi_data

def Convert_Midi_To_Xml(midi_data, midi_file_path, output_file_path):
    print(f"Converting MIDI to MusicXML: {midi_file_path} -> {output_file_path}")

    midi_data.save(midi_file_path)

    music_rep = converter.parse(midi_file_path)
    
    music_rep.write('musicxml', fp=output_file_path)
    print(f"MusicXML file created: {output_file_path}")
    return output_file_path


In [ ]:
def process_midi_file():

    os.makedirs("./resultMIDI", exist_ok=True)
    os.makedirs("./resultXML", exist_ok=True)

    midi_file_path = filedialog.askopenfilename(
        title="Select MIDI File to Process",
        filetypes=[("MIDI files", "*.mid"), ("All files", "*.*")]
    )
    
    if not midi_file_path:
        print("No file selected. Operation cancelled.")
        return

    midi_data = ParsingMidi_ForButton(midi_file_path)
    if midi_data is None:
        return

    processed_midi = PreprocessMidiData(midi_data)

    clean_midi = Filter_Data(processed_midi, model)

    output_midi_path = os.path.join("./resultMIDI", os.path.basename(midi_file_path))
    output_xml_path = os.path.join("./resultXML", 
                                  os.path.splitext(os.path.basename(midi_file_path))[0] + ".xml")

    xml_path = convert_midi_to_musicxml(clean_midi, output_midi_path, output_xml_path)

    result_label.config(text=f"Processing complete!\nMusicXML saved to: {xml_path}")

    open_folder_button.config(state=tk.NORMAL)
    global current_output_folder
    current_output_folder = os.path.dirname(xml_path)

In [ ]:
def Oopen_Output_Folder():
    import subprocess
    import platform
    
    if platform.system() == "Windows":
        os.startfile(current_output_folder)
    elif platform.system() == "Darwin":  # macOS
        subprocess.call(["open", current_output_folder])
    else:  # Linux
        subprocess.call(["xdg-open", current_output_folder])

In [ ]:
print("Starting model training...")
model_new = model
print("Model ready for use")


root = tk.Tk()
root.title("MIDI to MusicXML Converter")
root.geometry("500x300")


root.configure(padx=20, pady=20)


header = tk.Label(root, text="MIDI to MusicXML Converter", font=("Arial", 16, "bold"))
header.pack(pady=10)


instructions = tk.Label(root, text="Select a MIDI file to process and convert to MusicXML.")
instructions.pack(pady=5)


upload_button = tk.Button(root, text="Upload MIDI File", command=process_midi_file, 
                         font=("Arial", 12), padx=10, pady=5)
upload_button.pack(pady=20)


result_label = tk.Label(root, text="", wraplength=400)
result_label.pack(pady=10)


current_output_folder = ""
open_folder_button = tk.Button(root, text="Open Output Folder", command=open_output_folder,
                              state=tk.DISABLED, font=("Arial", 10))
open_folder_button.pack(pady=5)


status_label = tk.Label(root, text="Model loaded and ready", fg="green")
status_label.pack(pady=10)
root.mainloop()